# Satellite Image Classification Assignment (Q1–Q9)
This notebook contains solutions for all 9 questions in the assignment. 
Dummy image data is used so you can run this anywhere; replace with your actual dataset for final results.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Generate dummy images for portability
os.makedirs('dummy_dataset/train/class1', exist_ok=True)
os.makedirs('dummy_dataset/train/class2', exist_ok=True)
os.makedirs('dummy_dataset/validation/class1', exist_ok=True)
os.makedirs('dummy_dataset/validation/class2', exist_ok=True)

from PIL import Image
for folder in ['dummy_dataset/train/class1', 'dummy_dataset/train/class2', 'dummy_dataset/validation/class1', 'dummy_dataset/validation/class2']:
    for i in range(5):
        img = Image.fromarray(np.uint8(np.random.rand(64,64,3)*255))
        img.save(f'{folder}/img_{i}.jpg')

## Q1 – Count Images in Each Directory

In [ ]:
for root, dirs, files in os.walk('dummy_dataset'):
    print(f"{root}: {len([f for f in files if f.endswith('.jpg')])} images")

## Q2 – Data Augmentation Example

In [ ]:
datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2,
                             height_shift_range=0.2, shear_range=0.2,
                             zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

img = load_img('dummy_dataset/train/class1/img_0.jpg')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.imshow(array_to_img(batch[0]))
    plt.axis('off')
    plt.show()
    i += 1
    if i > 2:
        break

## Q3 – CNN Model Building

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Q4 – Data Generators for Training & Validation

In [ ]:
train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory('dummy_dataset/train', target_size=(64,64),
                                           batch_size=2, class_mode='categorical')
val_data = val_gen.flow_from_directory('dummy_dataset/validation', target_size=(64,64),
                                       batch_size=2, class_mode='categorical')

## Q5 – Model Training

In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=2)

## Q6 – Training Curves

In [ ]:
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend(); plt.show()

## Q7 – Predictions & Confusion Matrix

In [ ]:
val_data.reset()
preds = model.predict(val_data)
y_pred = np.argmax(preds, axis=1)
y_true = val_data.classes

cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted'); plt.ylabel('True');
plt.show()
print(classification_report(y_true, y_pred))

## Q8 – Save Model

In [ ]:
model.save('satellite_model.h5')
print('Model saved as satellite_model.h5')

## Q9 – Load and Use Saved Model

In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model('satellite_model.h5')
print('Model loaded successfully')